In [1]:
#Q-3

In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms
import tqdm as tqdm

In [2]:
d1= torchvision.datasets.MNIST(root='data',train=True,transform=torchvision.transforms.ToTensor(),download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16260310.40it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 459393.73it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4425210.40it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4715477.42it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
d1

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
d2= torchvision.datasets.MNIST(root='data',train=False,transform=torchvision.transforms.ToTensor(),download=True)

In [5]:
d2

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
train_loader=torch.utils.data.DataLoader(d1,batch_size=64)
test_loader=torch.utils.data.DataLoader(d2,batch_size=4)

In [9]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.n1=nn.Linear(784,256)
    self.n2=nn.Linear(256,64)
    self.n3=nn.Linear(64,10)
    self.relu=nn.ReLU()
    self.softmax=nn.Softmax()

  def forward(self,x):
    x=x.view(-1,784)
    self.x1=self.n1(x)
    self.x2=self.relu(self.x1)
    self.x3=self.n2(self.x2)
    self.x4=self.relu(self.x3)
    self.x5=self.n3(self.x4)
    self.x6=self.softmax(self.x5)
    return self.x6

  def backward(self,input_data,true_labels,predicted_output,learning_rate=0.001):
        one_hot_labels=torch.zeros((true_labels.size(0),10),device=true_labels.device)
        one_hot_labels.scatter_(1, true_labels.unsqueeze(1),1)
        softmax_output=F.softmax(predicted_output,dim=1)
        gradient_loss_output=(softmax_output[:true_labels.size(0)]-one_hot_labels)/true_labels.size(0)
        gradient_loss_hidden2=torch.matmul(gradient_loss_output, self.n3.weight)
        gradient_loss_pre_hidden2=gradient_loss_hidden2*(self.x4[:true_labels.size(0)] > 0).float()
        self.n3.weight.grad=torch.matmul(gradient_loss_output.t(), self.x4[:true_labels.size(0)])
        self.n3.bias.grad=gradient_loss_output.sum(0)
        gradient_loss_hidden1=torch.matmul(gradient_loss_pre_hidden2,self.n2.weight)
        gradient_loss_pre_hidden1=gradient_loss_hidden1*(self.x2[:true_labels.size(0)] > 0).float()
        self.n2.weight.grad=torch.matmul(gradient_loss_pre_hidden2.t(), self.x2[:true_labels.size(0)])
        self.n2.bias.grad=gradient_loss_pre_hidden2.sum(0)
        self.n1.weight.grad=torch.matmul(gradient_loss_pre_hidden1.t(),input_data.view(-1, 28*28)[:true_labels.size(0)])
        self.n1.bias.grad=gradient_loss_pre_hidden1.sum(0)

        # updating weights and biases using the calculated gradients and the learning rate
        with torch.no_grad():
            for param in self.parameters():
                param -= learning_rate*param.grad


In [10]:
epochs=10
model=Network()
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)
optimizer=optim.SGD(model.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

for epoch in tqdm.tqdm(range(epochs)):
  model.train()
  for data,labels in train_loader:
    data=data.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    output=model(data)
    loss=criterion(output[:labels.size(0)],labels)
    model.backward(data,labels,output,learning_rate=0.001)

  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

print("yeahh!")

 10%|█         | 1/10 [00:07<01:10,  7.87s/it]

Epoch 1/10, Loss: 2.2979


 20%|██        | 2/10 [00:14<00:59,  7.39s/it]

Epoch 2/10, Loss: 2.2946


 30%|███       | 3/10 [00:22<00:53,  7.58s/it]

...

Epoch 8/10, Loss: 2.3644


 90%|█████████ | 9/10 [01:08<00:07,  7.66s/it]

Epoch 9/10, Loss: 2.3010


100%|██████████| 10/10 [01:15<00:00,  7.54s/it]

Epoch 10/10, Loss: 2.3010
yeahh!
